In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
import serial

ser = serial.Serial()
ser.port = 'COM4'
ser.baudrate = 9600
ser.bytesize = 8
ser.parity = serial.PARITY_NONE
ser.stopbits = serial.STOPBITS_ONE


s1 = 0
s2 = 0
s3 = 0
s4 = 0
s5 = 0
s6 = 0
s7 = 0


cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
classifier = Classifier("k_model.h5")
 
    
offset = 20
imgSize = 300
 
labels = ["Forward", "Backward", "Right", "Left", "Stop", "Pick", "Drop"]
 
while True:
    success, img = cap.read()
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    try:
        if hands:
            hand = hands[0]
            x, y, w, h = hand['bbox']

            imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
            imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

            imgCropShape = imgCrop.shape

            aspectRatio = h / w

            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                imgResizeShape = imgResize.shape
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize
                prediction, index = classifier.getPrediction(imgWhite, draw=False)
                #print(prediction, index)

            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                imgResizeShape = imgResize.shape
                hGap = math.ceil((imgSize - hCal) / 2)

                imgWhite[hGap:hCal + hGap, :] = imgResize
                prediction, index = classifier.getPrediction(imgWhite, draw=False)

            cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                          (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
            cv2.putText(imgOutput, labels[index], (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
            cv2.rectangle(imgOutput, (x-offset, y-offset),
                          (x + w+offset, y + h+offset), (255, 0, 255), 4)
            
            print(labels[index])
            
            ######################################################################
            
            if labels[index]=='Forward':
                s1 += 1
                
            if labels[index]=='Backward':
                s2 += 1
                
            if labels[index]=='Right':
                s3 += 1
                
            if labels[index]=='Left':
                s4+= 1
                
            if labels[index]=='Stop':
                s5+= 1
                
            if labels[index]=='Pick':
                s6+= 1
                
            if labels[index]=='Drop':
                s7+= 1
            
            ##################################################################################
            
            if labels[index]=='Forward' and s1 == 50:
                a = b'B'
                ser.open()
                ser.write(a)
                ser.close()
                s1= 0
                s2 = 0
                s3 = 0
                s4 = 0
                s5 = 0
                s6 = 0
                s7 = 0

            elif labels[index]=='Backward' and s2 == 50:
                print('sending')
                a = b'A'
                ser.open()
                ser.write(a)
                ser.close()
                print('sent')
                s1 = 0
                s2  =0
                s3 = 0
                s4 = 0
                s5 = 0
                s6 = 0
                s7 = 0
            
            elif labels[index]=='Right' and s3 == 20:
                a = b'C'
                ser.open()
                ser.write(a)
                ser.close()
                s1 = 0
                s2 = 0
                s3 = 0
                s4 = 0
                s5 = 0
                s6 = 0
                s7 = 0
                
            elif labels[index]=='Left' and s4 == 20:
                print('')
                a = b'D'
                ser.open()
                ser.write(a)
                ser.close()
                s1 = 0
                s2 = 0
                s3 = 0
                s4 = 0
                s5 = 0
                s6 = 0
                s7 = 0
                
            elif labels[index]=='Stop' and s5 == 50:
                a = b'E'
                ser.open()
                ser.write(a)
                ser.close()
                s1 = 0
                s2 = 0
                s3 = 0
                s4 = 0
                s5 = 0
                s6 = 0
                s7 = 0
                
            elif labels[index]=='Pick' and s6 == 50:
                a = b'F'
                ser.open()
                ser.write(a)
                ser.close()
                s1 = 0
                s2 = 0
                s3 = 0
                s4 = 0
                s5 = 0
                s6 = 0
                s7 = 0
                
            elif labels[index]=='Drop' and s7 == 50:
                a = b'G'
                ser.open()
                ser.write(a)
                ser.close()
                s1 = 0
                s2 = 0
                s3 = 0
                s4 = 0
                s5 = 0
                s6 = 0
                s7 = 0
        
    except:      
        pass
 
    cv2.imshow("Image", imgOutput)
    k = cv2.waitKey(1)
    if k == 27:
        break

cv2.destroyAllWindows()